In [ ]:
import itertools as it
from pyspark import SparkContext
import sys

In [ ]:
sc = SparkContext()

In [ ]:
file = sc.textFile("../2.txt")

In [ ]:
def split_line(line):
    user, friends = line.split('\t')
    return (user, friends.split(','))

lines = file.map(lambda line: split_line(line))

In [ ]:
def get_mutal_friends(lines):
    friends = lines.flatMap(lambda line: [((line[0], friend), float('inf')) for friend in line[1]])
    possible_friends = lines.flatMap(lambda line: [(pair, 1) for pair in it.permutations(line[1], 2)])

    pairs = friends.union(possible_friends)
    pairs = pairs.reduceByKey(lambda x, y: x + y) \
                 .filter(lambda val: val[1] != float('inf'))
    return pairs

pairs = get_mutal_friends(lines)

In [ ]:
def transfor_loners(line):
    return (int(line[0]),())

loners = lines.map(transfor_loners)

In [ ]:
def transform_pair(line):
    return (int(line[0][0]), (int(line[0][1]), line[1]))

def get_top_ten_recomendations(line):
    return sorted(line, key = lambda x: (-x[1], x[0]))[:10]
    
result = pairs.map(transform_pair) \
              .groupByKey() \
              .union(loners) \
              .reduceByKey(lambda x,y: x) \
              .mapValues(get_top_ten_recomendations) \
              .sortByKey()

In [ ]:
def parse_result(line):
    user, recs = line[0], line[1]
    rec_list = [str(rec[0]) for rec in recs]
    return f'{user}\t{",".join(rec_list)}'

parsed_result = result.map(parse_result).collect()

In [ ]:
with open('resulta.txt', 'w') as outfile:
    outfile.write('\n'.join(parsed_result))

In [ ]:
sc.stop()